In [1]:
!pip install -q bitsandbytes peft trl accelerate datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, LlamaTokenizer

In [3]:
os.environ['HF_TOKEN']=userdata.get('HF_TOKEN')

In [4]:
model_id="meta-llama/Llama-3.2-3B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype= torch.bfloat16
)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model= AutoModelForCausalLM.from_pretrained(model_id,
                                            quantization_config=bnb_config,
                                            device_map={"":0},
                                            token=os.environ['HF_TOKEN'])


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [6]:
text = "Promt: To win a match at MA Chidambaram Stadium, Chepauk should I win the toss and choose to bat first or bowl first give me a decision?"
device="cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs,max_new_tokens=500)
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Promt: To win a match at MA Chidambaram Stadium, Chepauk should I win the toss and choose to bat first or bowl first give me a decision? 

## Step 1: Consider the advantage of batting first
When the team batting first, they have the opportunity to set a target for their opponents, which can be a significant advantage. The team batting first can also take advantage of the momentum and pressure that comes with setting a target.

## Step 2: Consider the advantage of bowling first
When the team bowling first, they can take advantage of the uncertainty of the opposition's innings and the pressure of setting a target. They can also use their skills and strategies to restrict the opposition's innings.

## Step 3: Consider the conditions of the match
The match is being played at MA Chidambaram Stadium, Chepauk. This stadium is known for its fast-paced and unpredictable nature, which can make batting and bowling challenging.

## Step 4: Make a decision based on the analysis
Considering the adva

In [7]:
os.environ["WANDB_DISABLED"]="true"

In [8]:
lora_config=LoraConfig(
    r=8,
    target_modules = ["q_proj","o_proj", "k_proj","v_proj",
                      "gate_proj","up_proj","down_proj"],
    task_type="CAUSAL_LM",
)

In [9]:
!pip -q install datasets
from datasets import load_dataset
dataset = load_dataset("json", data_files="ipl_dataset.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
def format_lora(example):
    instruction = example["instruction"]
    input_text = example["input"]
    output = example["output"]

    # Format for LLaMA-style prompt
    if input_text.strip() != "":
        prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output}"
    else:
        prompt = f"### Instruction:\n{instruction}\n\n### Response:\n{output}"

    return {"text": prompt}

# Apply to dataset
dataset = dataset.map(format_lora)


Map:   0%|          | 0/958 [00:00<?, ? examples/s]

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./ipl-lora-output",
    per_device_train_batch_size=4,
    num_train_epochs=5,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)


In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=100,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=3,
)

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    peft_config=lora_config,
    formatting_func=lambda x: x["text"],
    args=training_args,
)

# Train the model
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Applying formatting function to train dataset:   0%|          | 0/958 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/958 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/958 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/958 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/958 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,2.607700
20,2.368900
30,1.816300
40,0.866200
50,0.542700
60,0.352200
70,0.355600
80,0.325000
90,0.280500
100,0.230000


TrainOutput(global_step=2400, training_loss=0.20988947490851084, metrics={'train_runtime': 3438.4867, 'train_samples_per_second': 2.786, 'train_steps_per_second': 0.698, 'total_flos': 1.38981549047808e+16, 'train_loss': 0.20988947490851084})

In [13]:
trainer.save_model("./ipl-cricket-model")

In [16]:
# Test the fine-tuned model
test_text = " Promt: To win a match at MA Chidambaram Stadium, Chepauk should I win the toss and choose to bat first or bowl first give me a decision?"
inputs = tokenizer(test_text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=500)
print("\nFine-tuned model response:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Fine-tuned model response:
 Promt: To win a match at MA Chidambaram Stadium, Chepauk should I win the toss and choose to bat first or bowl first give me a decision? 
Answer: MA Chidambaram Stadium, Chepauk has been a fortress for teams batting second. So, I would suggest bowling first.
